In [1]:
import os
os.chdir('K:/ecole/github/televerser/civ6/ref/resources')
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
#get critical information from xml files
def get_info_from_xml(page,input_dict,tag_name,attr1,attr2):
    
    try:
        #get info name
        ftype=[row.get(attr1) for row in page.find(tag_name).find_all('row')]
        
        #get its attributes
        ttype=[row.get(attr2) for row in page.find(tag_name).find_all('row')]
    
        #due to weird structure of suitable terrains,features
        #the data structure has to be dictionary to host different values
        for ind,fname in enumerate(ftype):
            if fname not in input_dict:
                input_dict[fname]=[ttype[ind]]
            else:
                input_dict[fname]+=[ttype[ind]]
    except AttributeError:
        pass
    
    return input_dict

In [3]:
os.chdir('K:/ecole/github/televerser/civ6/ref/features')

#initial data structures
feature_list=[];feature_impassable=[];feature_tiles=[]
feature_nocoast=[];feature_noriver=[]
feature_validterrains={};feature_adjacentterrains={};feature_notadjacentterrains={}
feature_adjacentfeatures={};feature_notnearfeatures={}

#iterate through Australia_Features.xml; Expansion1_Features.xml;
#Expansion1_Features_Major.xml; Expansion2_Features.xml;
#Features.xml; GranColombia_Maya_Expansion2.xml;
#GranColombia_Maya_Features.xml; Indonesia_Khmer_GameplayData.xml;
#VikingsLandmarks_Features.xml
for filename in os.listdir():
    
    #read xml
    f=open(filename,'r')
    text=f.read()
    f.close()
    page=bs(text,'lxml')
    
    #get simple attrs
    feature_names=[i.get('type') for i in page.find('types').find_all('row')]
    feature_list+=[i for i in feature_names if 'feature' in i.lower()]
    feature_impassable+=[i.get('impassable') for i in page.find('features').find_all('row')]
    feature_tiles+=[i.get('tiles') for i in page.find('features').find_all('row')]
    feature_nocoast+=[i.get('nocoast') for i in page.find('features').find_all('row')]
    feature_noriver+=[i.get('noriver') for i in page.find('features').find_all('row')]
    
    #get complex attrs
    feature_validterrains=get_info_from_xml(page,feature_validterrains,
                                            'feature_validterrains',
                                            'featuretype','terraintype')   
    feature_adjacentterrains=get_info_from_xml(page,feature_adjacentterrains,
                                            'feature_adjacentterrains',
                                            'featuretype','terraintype')
    feature_notadjacentterrains=get_info_from_xml(page,feature_notadjacentterrains,
                                            'feature_adjacentterrains',
                                            'featuretype','terraintype')
    feature_adjacentfeatures=get_info_from_xml(page,feature_adjacentfeatures,
                                            'feature_adjacentfeatures',
                                            'featuretype','featuretypeadjacent')
    feature_notnearfeatures=get_info_from_xml(page,feature_notnearfeatures,
                                            'feature_notnearfeatures',
                                            'featuretype','featuretypeavoid')

In [4]:
#construct dataframe
feature_df=pd.DataFrame()
feature_df['name']=feature_list
feature_df['impassable']=feature_impassable
feature_df['tiles']=feature_tiles
feature_df['nocoast']=feature_nocoast
feature_df['noriver']=feature_noriver
feature_df['validterrains']=feature_df['name'].apply(lambda x:'' if x not in feature_validterrains else ';'.join(feature_validterrains[x]))
feature_df['adjacentterrains']=feature_df['name'].apply(lambda x:'' if x not in feature_adjacentterrains else ';'.join(feature_adjacentterrains[x]))
feature_df['notadjacentterrains']=feature_df['name'].apply(lambda x:'' if x not in feature_notadjacentterrains else ';'.join(feature_notadjacentterrains[x]))
feature_df['adjacentfeatures']=feature_df['name'].apply(lambda x:'' if x not in feature_adjacentfeatures else ';'.join(feature_adjacentfeatures[x]))
feature_df['notnearfeatures']=feature_df['name'].apply(lambda x:'' if x not in feature_notnearfeatures else ';'.join(feature_notnearfeatures[x]))

#fillna
feature_df['noriver']=feature_df['noriver'].fillna('false')
feature_df['nocoast']=feature_df['nocoast'].fillna('false')

# #output
# feature_df.drop_duplicates().to_csv('feature mapping.csv',index=False)